### Working with Different Types of Data

In [ ]:
../data/retail-data/by-day/2010-12-01.csv

In [1]:
from pyspark import SparkContext, SQLContext
sc = SparkContext()
spark = SQLContext(sc)

In [2]:
df = spark.read.csv(path='../data/retail-data/by-day/2010-12-01.csv',header=True,inferSchema=None)

In [5]:
df.createOrReplaceTempView("dfTable")

In [9]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



In [15]:
from pyspark.sql.functions import *

df.select(lit(5), lit("five"), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

#### Working with Booleans

In [21]:
df.where(col("InvoiceNo") != 536365).select("InvoiceNo","Description").show(5,False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [25]:
priceFilter = col("UnitPrice") > 600
descFilter = col("Description").contains("POSTAGE")
df.where(df.StockCode.isin("DOT")).where(priceFilter|descFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [26]:
df.where(df.StockCode.isin("DOT")).where(priceFilter & descFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [39]:
DOTFilter = col("StockCode") == "DOT"
descFilter = instr(col("Description"),"POSTAGE") >= 1
df.withColumn("isExpensive", DOTFilter & (priceFilter|descFilter))\
.where("isExpensive").select("InvoiceNo","Description", "UnitPrice","isExpensive").show()

+---------+--------------+---------+-----------+
|InvoiceNo|   Description|UnitPrice|isExpensive|
+---------+--------------+---------+-----------+
|   536544|DOTCOM POSTAGE|   569.77|       true|
|   536592|DOTCOM POSTAGE|   607.49|       true|
+---------+--------------+---------+-----------+



#### Working With Numbers

In [45]:
fabricatedQuantity = pow((col("Quantity") * col("UnitPrice")),2) + 5
df.select("CustomerID", fabricatedQuantity.alias("RealQuantity")).show(2)

+----------+------------------+
|CustomerID|      RealQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [48]:
df.selectExpr("CustomerID", "POW((Quantity * UnitPrice), 2) + 5 as RealQuantity").show(2)

+----------+------------------+
|CustomerID|      RealQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [52]:
df.select(round(lit("2.5")).alias("Round"), bround(lit("2.5")).alias("BottonRound")).show(2)

+-----+-----------+
|Round|BottonRound|
+-----+-----------+
|  3.0|        2.0|
|  3.0|        2.0|
+-----+-----------+
only showing top 2 rows



In [55]:
df.select(corr("Quantity", "UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



In [62]:
df.describe("Quantity","UnitPrice","CustomerID").show()

+-------+------------------+------------------+------------------+
|summary|          Quantity|         UnitPrice|        CustomerID|
+-------+------------------+------------------+------------------+
|  count|              3108|              3108|              1968|
|   mean| 8.627413127413128| 4.151946589446603|15661.388719512195|
| stddev|26.371821677029203|15.638659854603892|1854.4496996893627|
|    min|                -1|               0.0|           12431.0|
|    max|                96|              9.95|           18229.0|
+-------+------------------+------------------+------------------+



+-------+------------------+
|summary|         StockCode|
+-------+------------------+
|  count|              3108|
|   mean|27834.304044117645|
| stddev|17407.897548583845|
|    min|             10002|
|    max|              POST|
+-------+------------------+



#### Working wiht String

In [68]:
df.select(initcap(col("Description")),\
          upper(col("Description")),\
          lower(col("Description"))).show(2,False)

+----------------------------------+----------------------------------+----------------------------------+
|initcap(Description)              |upper(Description)                |lower(Description)                |
+----------------------------------+----------------------------------+----------------------------------+
|White Hanging Heart T-light Holder|WHITE HANGING HEART T-LIGHT HOLDER|white hanging heart t-light holder|
|White Metal Lantern               |WHITE METAL LANTERN               |white metal lantern               |
+----------------------------------+----------------------------------+----------------------------------+
only showing top 2 rows



In [81]:
df.select(\
    ltrim(lit("    HELLO    ")).alias("ltrim"),\
    rtrim(lit("    HELLO    ")).alias("rtrim"),\
    trim(lit("    HELLO    ")).alias("trim"),\
    lpad(lit("HELLO"), 10, "*").alias("lp"),\
    rpad(lit("HELLO"), 10, "*").alias("rp")).show(3)

+---------+---------+-----+----------+----------+
|    ltrim|    rtrim| trim|        lp|        rp|
+---------+---------+-----+----------+----------+
|HELLO    |    HELLO|HELLO|*****HELLO|HELLO*****|
|HELLO    |    HELLO|HELLO|*****HELLO|HELLO*****|
|HELLO    |    HELLO|HELLO|*****HELLO|HELLO*****|
+---------+---------+-----+----------+----------+
only showing top 3 rows



#### Regular Expression

In [101]:
color_string = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(regexp_replace(col("Description"),color_string,'COLOR').alias("CleanCode")).show(3,False)

+----------------------------------+
|CleanCode                         |
+----------------------------------+
|COLOR HANGING HEART T-LIGHT HOLDER|
|COLOR METAL LANTERN               |
|CREAM CUPID HEARTS COAT HANGER    |
+----------------------------------+
only showing top 3 rows



In [92]:
df.select(translate(col("Description"),"LEET","1337"), col("Description")).show(3, False)

+----------------------------------+----------------------------------+
|translate(Description, LEET, 1337)|Description                       |
+----------------------------------+----------------------------------+
|WHI73 HANGING H3AR7 7-1IGH7 HO1D3R|WHITE HANGING HEART T-LIGHT HOLDER|
|WHI73 M37A1 1AN73RN               |WHITE METAL LANTERN               |
|CR3AM CUPID H3AR7S COA7 HANG3R    |CREAM CUPID HEARTS COAT HANGER    |
+----------------------------------+----------------------------------+
only showing top 3 rows



In [103]:
color_string = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(regexp_extract(col("Description"),color_string,1)).show(5)

+------------------------------------------------------------+
|regexp_extract(Description, (BLACK|WHITE|RED|GREEN|BLUE), 1)|
+------------------------------------------------------------+
|                                                       WHITE|
|                                                       WHITE|
|                                                            |
|                                                            |
|                                                         RED|
+------------------------------------------------------------+
only showing top 5 rows



In [110]:
# find description that has black or white in description
containBlack = instr(col("Description"),'BLACK') >= 1
containWhite = instr(col("Description"),'WHITE') >= 1
df.withColumn('hasSimpleColor',containBlack|containWhite).\
where("hasSimpleColor").select("Description").show(3,False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



#### Working with Date and timeStamp

In [134]:
date_df = spark.range(10)\
.withColumn("today",current_date())\
.withColumn("now", current_timestamp())

date_df.show(3,False)

date_df.createOrReplaceTempView("dateTable")

+---+----------+-----------------------+
|id |today     |now                    |
+---+----------+-----------------------+
|0  |2019-05-25|2019-05-25 21:17:05.403|
|1  |2019-05-25|2019-05-25 21:17:05.403|
|2  |2019-05-25|2019-05-25 21:17:05.403|
+---+----------+-----------------------+
only showing top 3 rows



In [135]:
date_df.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



In [136]:
date_df.select(date_sub(col("today"),5), date_add(col("now"),5)).show(3)

+------------------+----------------+
|date_sub(today, 5)|date_add(now, 5)|
+------------------+----------------+
|        2019-05-20|      2019-05-30|
|        2019-05-20|      2019-05-30|
|        2019-05-20|      2019-05-30|
+------------------+----------------+
only showing top 3 rows



In [137]:
# datediff, months_between, to_date

In [153]:
df.select(to_date(lit("2019-05-20")).alias("start"),\
          to_date(lit("2019-08-30")).alias("end"))\
.withColumn("diff",datediff(col("start"), col("end")))\
.withColumn('month_between', months_between(col("end"), col("start")))\
.show(3)

+----------+----------+----+-------------+
|     start|       end|diff|month_between|
+----------+----------+----+-------------+
|2019-05-20|2019-08-30|-102|   3.32258065|
|2019-05-20|2019-08-30|-102|   3.32258065|
|2019-05-20|2019-08-30|-102|   3.32258065|
+----------+----------+----+-------------+
only showing top 3 rows



In [170]:
clean_date_df = spark.range(3).withColumn('date',to_date(lit("2019-21-08"),"yyyy-dd-MM"))
clean_date_df.show()

+---+----------+
| id|      date|
+---+----------+
|  0|2019-08-21|
|  1|2019-08-21|
|  2|2019-08-21|
+---+----------+



In [172]:
clean_date_df.select(to_timestamp(col('date'))).show()

+--------------------+
|to_timestamp(`date`)|
+--------------------+
| 2019-08-21 00:00:00|
| 2019-08-21 00:00:00|
| 2019-08-21 00:00:00|
+--------------------+



In [175]:
clean_date_df.select(to_utc_timestamp(col('date'),'EST')).show()

+---------------------------+
|to_utc_timestamp(date, EST)|
+---------------------------+
|        2019-08-21 05:00:00|
|        2019-08-21 05:00:00|
|        2019-08-21 05:00:00|
+---------------------------+



#### Working with Null

In [181]:
df.select(coalesce(col("Description")), coalesce(col("CustomerID"))).show(3,False)

+----------------------------------+--------------------+
|coalesce(Description)             |coalesce(CustomerID)|
+----------------------------------+--------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|17850.0             |
|WHITE METAL LANTERN               |17850.0             |
|CREAM CUPID HEARTS COAT HANGER    |17850.0             |
+----------------------------------+--------------------+
only showing top 3 rows



In [183]:
df.na.drop().show(1)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row



In [185]:
df.na.drop("any").count()

1968

In [186]:
df.na.drop("all").count()

3108

In [188]:
df.na.drop("all",subset=["StockCode", "InvoiceNo"]).count()

3108

In [191]:
df.na.fill("All Null values become this string").show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



In [192]:
df.na.fill("all", subset=["StockCode", "InvoiceNo"])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: string, InvoiceDate: string, UnitPrice: string, CustomerID: string, Country: string]

In [193]:
df.na.replace([""], ["UNKNOWN"], "Description")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: string, InvoiceDate: string, UnitPrice: string, CustomerID: string, Country: string]